In [3]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('../../../code/')
import leakage
import utils
import FBA_leak
from pathlib import Path

In [2]:
from dotenv import find_dotenv

In [4]:
model = cobra.io.read_sbml_model('../../../models/e_coli/momentiJO1366.xml')
# model = cobra.io.read_sbml_model('../../models/e_coli/iJO1366.xml')

# model = cobra.io.read_sbml_model('../../models/e_coli/iJR904.xml')
# model = cobra.io.read_sbml_model('../../models/e_coli/e_coli_core.xml')

model.solver = 'gurobi'

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [9]:
model = cobra.io.read_sbml_model('../../../models/e_coli/momentiJO1366.xml')



<Solution 0.743 at 0x7fa9082db8e0>


In [15]:
model.reactions.EX_glc__D_e.lower_bound = -10
print(model.optimize())
model.reactions.EX_ac_e.lower_bound = 0
print(model.summary())

<Solution 0.704 at 0x7fa9289f7eb0>
Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.704473431869784

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
 prot_pool  ER_pool_TG_     0.095         0   0.00%
     ca2_e     EX_ca2_e  0.003667         0   0.00%
      cl_e      EX_cl_e  0.003667         0   0.00%
 cobalt2_e EX_cobalt2_e 1.761E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0004995         0   0.00%
     fe2_e     EX_fe2_e   0.01131         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1375         0   0.00%
     mg2_e     EX_mg2_e  0.006111         0   0.00%
     mn2_e     EX_mn2_e 0.0004868         0   0.00%
    mobd_e    EX_mobd_e 9.088E-05         0   0.00%
     nh4_e     EX_nh4_e     7.609         0   0.00%
     ni2_e     EX_ni2_e 0.0002275         0   0.00%
      o2_e      EX_o2_e     19.86         0   0.00%
      pi_e      EX_pi_e    0.6796         0   0.00%
     so4_e     EX_so4_e    0.1777         0   0.00%
 

In [20]:
model.problem.Objective((model.reactions.ABTA.forward_variable- 3)**2+(model.reactions.APCS.forward_variable- 3)**2)

In [21]:
exp = (model.reactions.ABTA.forward_variable- 3)**2

In [8]:
print(model.optimize())
model.summary()

<Solution 0.743 at 0x7fa94ae480d0>


Metabolite,Reaction,Flux,C-Number,C-Flux
prot_pool,ER_pool_TG_,0.095,0,0.00%
ca2_e,EX_ca2_e,0.003867,0,0.00%
cl_e,EX_cl_e,0.003867,0,0.00%
cobalt2_e,EX_cobalt2_e,1.857E-05,0,0.00%
cu2_e,EX_cu2_e,0.0005268,0,0.00%
fe2_e,EX_fe2_e,0.01193,0,0.00%
glc__D_e,EX_glc__D_e,13.83,6,100.00%
k_e,EX_k_e,0.145,0,0.00%
mg2_e,EX_mg2_e,0.006445,0,0.00%
mn2_e,EX_mn2_e,0.0005134,0,0.00%


In [7]:
with model as M:
    m = M.metabolites.icit_c
    m_e = m.copy()
    m_e.id = m.id.replace('_c','_e')
    m_e.compartment = 'e'
    M.add_boundary(m_e, type='exchange')
    print([r.id for r in M.exchanges])

['EX_h2o_e', 'EX_12ppd__S_e', 'EX_15dap_e', 'EX_26dap__M_e', 'EX_2ddglcn_e', 'EX_3hcinnm_e', 'EX_3hpppn_e', 'EX_4abut_e', 'EX_ac_e', 'EX_acac_e', 'EX_h_e', 'EX_acald_e', 'EX_acgam_e', 'EX_acmana_e', 'EX_hdca_e', 'EX_acnam_e', 'EX_ade_e', 'EX_his__L_e', 'EX_adn_e', 'EX_akg_e', 'EX_hxan_e', 'EX_ala__D_e', 'EX_idon__L_e', 'EX_ala__L_e', 'EX_alltn_e', 'EX_amp_e', 'EX_arab__L_e', 'EX_arg__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_ile__L_e', 'EX_but_e', 'EX_cbl1_e', 'EX_chol_e', 'EX_indole_e', 'EX_cit_e', 'EX_co2_e', 'EX_crn_e', 'EX_csn_e', 'EX_ins_e', 'EX_cynt_e', 'EX_k_e', 'EX_cys__L_e', 'EX_cytd_e', 'EX_lac__D_e', 'EX_dad_2_e', 'EX_dcyt_e', 'EX_dgsn_e', 'EX_dha_e', 'EX_din_e', 'EX_dms_e', 'EX_lac__L_e', 'EX_dmso_e', 'EX_duri_e', 'EX_etoh_e', 'EX_lcts_e', 'EX_fe2_e', 'EX_for_e', 'EX_leu__L_e', 'EX_fru_e', 'EX_lys__L_e', 'EX_fuc1p__L_e', 'EX_fuc__L_e', 'EX_fum_e', 'EX_mal__L_e', 'EX_g6p_e', 'EX_gal_e', 'EX_galct__D_e', 'EX_galctn__D_e', 'EX_galt_e', 'EX_malt_e', 'EX_galur_e', 'EX_gam_e', 'EX_

In [8]:
# model.reactions.DHAPT.knock_out()
# model.reactions.THRA.knock_out()
# model.reactions.THRA2.knock_out()
# model.reactions.F6PA_TG_forward.knock_out()

In [4]:
# Initial conditions is 0.013 gDW/L (in total)
model_name = 'Ecoli'
model_name_dict = {model_name: [model, 0.006]}

glucose_mM = utils.convert_gL_to_mM("C6H12O6", 4)
D = FBA_leak.dFBA(iterations = 3, dt = 0.1, method = "FBA_with_leakage", store_exchanges_flag = False)#(, pfba_fraction = 0.95)
D.add_models(model_name_dict)

##### Lekage
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpa09j3b3o.lp
Reading time = 0.00 seconds
: 761 rows, 2150 columns, 9006 nonzeros


In [5]:
# Set Km and vMax
D.models[model_name].set_km("glc__D_e", 1)
D.models[model_name].set_Vmax("glc__D_e", 10)



D.medium.define_initial_conditions({"glc__D_e": glucose_mM})
# D.medium.set_store_concentrations(["glc__D_e", "nh3_e"])
D.run()
print(D.biomass_df)

dFBA simulation took 10 seconds
   Timepoint  Time     Ecoli
0          0   0.0  0.006000
1          1   0.1  0.006524


# Make auxotrophs
Using data from Mee et al., 2014 https://www.pnas.org/doi/epdf/10.1073/pnas.1405641111
R: argA,
C: cysE,
G: glyA,
H: hisB,
I: ilvA,
L: leuB,
K: lysA,
M: metA,
F: pheA,
P: proA,
S: serA,
T: thrC,
W: trpC,
Y: tyrA

In [4]:
KO_strain_dict = {
'R': 'argA',
'C': 'cysE',
'G': 'glyA',
'H': 'hisB',
'I': 'ilvA',
'L': 'leuB',
'K': 'lysA',
'M': 'metA',
'F': 'pheA',
'P': 'proA',
'S': 'serA',
'T': 'thrC',
'W': 'trpC',
'Y': 'tyrA'
}

In [2]:
# model = cobra.io.read_sbml_model('../../models/e_coli/iJR904.xml')
model = cobra.io.read_sbml_model('../../models/e_coli/iJO1366.xml')
# model = cobra.io.read_sbml_model('../../models/e_coli/e_coli_core.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-26


In [5]:
folder = Path('../../models/e_coli/ko/')


model.solver = 'gurobi'
model_dict = {}
for key, gene_name in KO_strain_dict.items():
    with model as M:
        genes = [g for g in M.genes if g.name == gene_name]
        rxns = cobra.manipulation.knock_out_model_genes(M, genes)
        if key == 'I':
            # According to the model, tdcB can perform the same function, but we assume this reaction is knocked out anyway here
            M.reactions.THRD_L.bounds = (0,0)
        if key =='G':
            M.reactions.GLYAT.bounds = (0,1000)
            M.reactions.THRA.bounds = (0,0)
            M.reactions.THRA2.bounds = (0,0)
        if key =='P':
            # model.reactions.NACODA.bounds = (0,0)
            M.reactions.P5CR.bounds = (0,0)
        if key =='S':
            M.reactions.GHMT2r.bounds = (0,1000)
        if key =='T':
            M.reactions.THRA.bounds = (0,1000)
        print(key, genes[0], M.slim_optimize())
        # print(M.metabolites.ser__L_c.summary())
        cobra.io.write_sbml_model(M, folder/'{0}.xml'.format(key))
        model_dict[key] = M

R b2818 0.0
C b3607 0.0
G b2551 0.0
H b2022 0.0
I b3772 0.0
L b0073 0.0
K b2838 0.0
M b4013 0.0
F b2599 0.0
P b0243 0.0
S b2913 0.0
T b0004 0.0
W b1262 0.0
Y b2600 0.0


In [28]:
model.optimize()
print(cobra.flux_analysis.pfba(model))
print(model.summary())

<Solution 699.022 at 0x7fd179c638b0>
Objective
1.0 BIOMASS_Ec_iJO1366_core_53p95M = 0.9823718127269754

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e  0.005113         0   0.00%
      cl_e      EX_cl_e  0.005113         0   0.00%
 cobalt2_e EX_cobalt2_e 2.456E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0006965         0   0.00%
     fe2_e     EX_fe2_e   0.01578         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1918         0   0.00%
     mg2_e     EX_mg2_e  0.008522         0   0.00%
     mn2_e     EX_mn2_e 0.0006788         0   0.00%
    mobd_e    EX_mobd_e 0.0001267         0   0.00%
     nh4_e     EX_nh4_e     10.61         0   0.00%
     ni2_e     EX_ni2_e 0.0003173         0   0.00%
      o2_e      EX_o2_e     17.58         0   0.00%
      pi_e      EX_pi_e    0.9476         0   0.00%
     so4_e     EX_so4_e    0.2478         0   0.00%
     zn2_e     EX_zn2_e  0.000335         0   0.00

In [17]:
s = cobra.flux_analysis.pfba(model)

In [20]:
s

,fluxes,reduced_costs
EX_cm_e,0.000000,-2.0
EX_cmp_e,0.000000,178.2
EX_co2_e,19.675223,-2.0
EX_cobalt2_e,-0.000025,2.0
DM_4crsol_c,0.000219,-2.0
...,...,...
RNDR4,0.000000,-2.0
RNDR4b,0.000000,-2.0
RNTR1c2,0.025705,-2.0
RNTR2c2,0.026541,-2.0


# Simulate auxotrophs

In [6]:
model_name = 'Ecoli'
model_name_dict = {model_name: [model_dict['I'], 0.006]}

glucose_mM = utils.convert_gL_to_mM("C6H12O6", 4)
D = FBA_leak.dFBA(iterations = 3, dt = 0.1, method = "FBA_with_leakage", store_exchanges_flag = False)#(, pfba_fraction = 0.95)
D.add_models(model_name_dict)
# Set Km and vMax
D.models[model_name].set_km("glc__D_e", 1)
D.models[model_name].set_Vmax("glc__D_e", 10)



D.medium.define_initial_conditions({"glc__D_e": glucose_mM, "ile__L_e": 1})
D.run()
print(D.biomass_df)

##### Lekage
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpoc1bkvlp.lp
Reading time = 0.00 seconds
: 761 rows, 2150 columns, 9006 nonzeros
dFBA simulation took 10 seconds
   Timepoint  Time     Ecoli
0          0   0.0  0.006000
1          1   0.1  0.006543


In [22]:
D.concentrations_df[['glc__D_e', 'ile__L_e']]

,glc__D_e,ile__L_e
0,22.202994,1.000000
1,22.196454,0.999836
2,22.189321,0.999658


In [10]:
for x in D.concentrations_df.columns:
    m = model.metabolites.get_by_id(x.replace("_e", "_c"))
    print(m.id, m.formula_weight, D.concentrations_df[x].max())

10fthf_c 471.42344 5.716009444975999e-07
13dpg_c 262.005382 1.0486222312436954e-05
1pyr5c_c 112.10664 4.731233673722961e-06
23dhdp_c 167.1189 3.727553324274676e-06
23dhmb_c 133.12256 5.010496238288017e-06
25aics_c 450.251561 1.3022634009434882e-06
26dap_LL_c 190.19706 2.925990038853849e-06
2cpr5p_c 346.206681 1.213939193873706e-06
2dda7p_c 285.12206100000003 3.2569500418461468e-06
2dr1p_c 212.09452100000001 3.892126247839106e-06
2dr5p_c 212.09452100000001 3.892126247839044e-06
2ippm_c 156.13602 3.233828574374865e-06
2pg_c 183.033421 1.206470536637243e-05
34hpp_c 179.14948 2.0334649037058393e-06
3c2hmp_c 174.1513 3.2338285743755803e-06
3c3hmp_c 174.1513 3.2338285743756684e-06
3c4mop_c 172.13542 3.53137770839612e-06
3dhq_c 189.14276 3.3040578304744168e-06
3dhsk_c 171.12748 3.304057830475455e-06
3ig3p_c 285.189841 1.2224728030777829e-06
3mob_c 115.10728 5.010496238281086e-06
3pg_c 183.033421 1.206470536637695e-05
3php_c 181.017541 1.5357361845737776e-05
3psme_c 320.146221 1.87912485007650

KeyError: 'Timepoint'

In [29]:
print(D.concentrations_df.max().sort_values())

u23ga_e         1.804664e-07
lipidX_e        2.295764e-07
ugmda_e         2.617167e-07
peptido_EC_e    2.651231e-07
u3aga_e         3.111130e-07
                    ...     
na1_e           1.000000e+03
nh4_e           1.000000e+03
o2_e            1.000000e+03
btn_e           1.000000e+03
pi_e            1.000000e+03
Length: 198, dtype: float64


In [ ]:
[x for x in D.concentrations_df.columns if D]

In [6]:
r = model.reactions.EX_12ppd__R_e

In [15]:
r.metabolites.popitem()[0].id

'12ppd__R_e'

# Yield of AA

In [49]:
model.reactions.BIOMASS_Ecoli_core_w_GAM

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x7fe581d94880
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0
